In [ ]:
!git clone https://github.com/Dt-Pham/Advanced-Lane-Lines.git
%cd Advanced-Lane-Lines/

# Cloning into 'Advanced-Lane-Lines'...
# remote: Enumerating objects: 104, done.
# remote: Counting objects: 100% (4/4), done.
# remote: Compressing objects: 100% (4/4), done.
# remote: Total 104 (delta 3), reused 0 (delta 0), pack-reused 100
# Receiving objects: 100% (104/104), 95.77 MiB | 12.09 MiB/s, done.
# Resolving deltas: 100% (6/6), done.
# /content/Advanced-Lane-Lines


In [ ]:
!pip install docopt==0.6.2

# Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
# Collecting docopt==0.6.2
#   Downloading docopt-0.6.2.tar.gz (25 kB)
#   Preparing metadata (setup.py) ... done
# Building wheels for collected packages: docopt
#   Building wheel for docopt (setup.py) ... done
#   Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13721 sha256=008c57e3a2edb43eec03f701e71270a44ca38b0639464a103d238666f174a38e
#   Stored in directory: /root/.cache/pip/wheels/70/4a/46/1309fc853b8d395e60bafaf1b6df7845bdd82c95fd59dd8d2b
# Successfully built docopt
# Installing collected packages: docopt
# Successfully installed docopt-0.6.2

In [ ]:
import numpy as np
import matplotlib.image as mpimg
import cv2
from docopt import docopt
from IPython.display import HTML
from IPython.core.display import Video
from moviepy.editor import VideoFileClip
from CameraCalibration import CameraCalibration
from Thresholding import *
from PerspectiveTransformation import *
from LaneLines import *

class FindLaneLines:
    def __init__(self):
        """ Init Application"""
        self.calibration = CameraCalibration('camera_cal', 9, 6)
        self.thresholding = Thresholding()
        self.transform = PerspectiveTransformation()
        self.lanelines = LaneLines()

    def forward(self, img):
        out_img = np.copy(img)
        img = self.calibration.undistort(img)
        img = self.transform.forward(img)
        img = self.thresholding.forward(img)
        img = self.lanelines.forward(img)
        img = self.transform.backward(img)

        out_img = cv2.addWeighted(out_img, 1, img, 0.6, 0)
        out_img = self.lanelines.plot(out_img)
        return out_img

    def process_image(self, input_path, output_path):
        img = mpimg.imread(input_path)
        out_img = self.forward(img)
        mpimg.imsave(output_path, out_img)

    def process_video(self, input_path, output_path):
        clip = VideoFileClip(input_path)
        out_clip = clip.fl_image(self.forward)
        out_clip.write_videofile(output_path, audio=False)

def main():
    findLaneLines = FindLaneLines()
    findLaneLines.process_video("challenge_video.mp4","output.mp4")

if __name__ == "__main__":
    main()